# WEEK 6
## Encounter 02 - SQL with Python
## Project Challenge - Countries Data

### Task Description
The table `country_codes.csv` contains a list of countries and their numeric, two and three letters country codes. Also it includes latitude and longitude coordinates of the geographic center of each country. This table will be useful later in the week when visualizing country level data on a map.

   1. Import the csv file as a pandas data frame.
   2. With Python, define a **countries table** in the **climate database**:
   3. Load the data frame into the **countries table**

In [5]:
import pandas as pd
import os
from dotenv import load_dotenv

from sqlalchemy import create_engine
from sqlalchemy import text  # to be able to pass string

In [3]:
# load variables for login from .env file
load_dotenv()

True

In [4]:
# get values of variables for the login
username = os.getenv('USER')
password = os.getenv('PASS')
host = os.getenv('HOST')
port = os.getenv('PORT')

In [6]:
# composing a connection string for postgresql, 'climate' database
url = f'postgresql://{username}:{password}@{host}:{port}/climate'

In [7]:
# create a connection engine
engine = create_engine(url, echo=False)

In [8]:
# crete table 
# "name","alpha2","alpha3","code","lat","lon"
with engine.begin() as conn:
    conn.execute(text("DROP TABLE IF EXISTS countries CASCADE;"))
    conn.execute(text("""
        CREATE TABLE countries (
            name VARCHAR,
            alpha2 VARCHAR(2) PRIMARY KEY,
            alpha3 VARCHAR(3),
            code INT,
            lat NUMERIC,
            lon NUMERIC
        );
    """))

In [11]:
# reading the data from csv-file

countries = pd.read_csv('../project_data/country_codes.csv')
countries                  

,name,alpha2,alpha3,code,lat,lon
0,Afghanistan,AF,AFG,4,33.0000,65.0
1,Albania,AL,ALB,8,41.0000,20.0
2,Algeria,DZ,DZA,12,28.0000,3.0
3,American Samoa,AS,ASM,16,-14.3333,-170.0
4,Andorra,AD,AND,20,42.5000,1.6
...,...,...,...,...,...,...
238,Wallis and Futuna,WF,WLF,876,-13.3000,-176.2
239,Western Sahara,EH,ESH,732,24.5000,-13.0
240,Yemen,YE,YEM,887,15.0000,48.0
241,Zambia,ZM,ZMB,894,-15.0000,30.0


In [12]:
# loading 'countries' data from DataFrame to 'countries' table in 'climate' database
countries.to_sql('countries', engine, if_exists='append', index=False)

243

## Project Challenge - Stations Data

### Task Description

Using the stations file found in the downloaded ECA_blend data folder proceed with it in the same way as for the countries data from the previous exercise. Use the downloaded stations file as it will have all stations found in the downloaded datasets.

**Hints:**

  * First read stations data into a notebook and clean up before uploading to database
  * The pd.read_csv method has a skiprows parameter to skip some header lines of a .csv file
  * You need to cleanup the column names of the file. Watch out for whitespace and convert the names to lowercase
  * Add a foreign key constraint for the cn column and let it point to the alpha2 column of the countries table

In [13]:
# read 'stations.txt' file
stations = pd.read_csv('../project_data/stations.txt', skiprows=17)
stations

,STAID,STANAME,CN,LAT,LON,HGHT
0,1,VAEXJOE,SE,+56:52:00,+014:48:00,166
1,2,FALUN,SE,+60:37:00,+015:37:00,160
2,3,STENSELE,SE,+65:04:00,+017:09:59,325
3,4,LINKOEPING,SE,+58:24:00,+015:31:59,93
4,5,LINKOEPING-MALMSLAETT,SE,+58:24:00,+015:31:59,93
...,...,...,...,...,...,...
6450,25150,GDANSK-REBIECHOWO_OLD,PL,+54:22:59,+018:28:00,144
6451,25151,ELBLAG-MILEJEWO,PL,+54:13:23,+019:32:36,151
6452,25156,KROSNO,PL,+49:42:24,+021:46:09,326
6453,25157,YLJA KRAFTVERK,NO,+61:11:49,+008:22:50,517


### Stations data file description:
FILE FORMAT (MISSING VALUE CODE IS -9999):

STAID  : Station identifier  
STANAME: Station name  
CN     : Country code (ISO3116 countrycodes)  
LAT    : Latitude in degrees:minutes:seconds (+: North, -: South)  
LON    : Longitude in degrees:minutes:seconds (+: East, -: West)  
HGTH   : Station elevation in meters  

In [14]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6455 entries, 0 to 6454
Data columns (total 6 columns):
 #   Column                                    Non-Null Count  Dtype 
---  ------                                    --------------  ----- 
 0   STAID                                     6455 non-null   int64 
 1   STANAME                                   6455 non-null   object
 2   CN                                        6455 non-null   object
 3         LAT                                 6455 non-null   object
 4          LON                                6455 non-null   object
 5   HGHT                                      6455 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 302.7+ KB


In [19]:
# cleaning up the column names
stations.columns = ['staid', 'staname', 'cn', 'lat', 'lon', 'hght']

In [22]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6455 entries, 0 to 6454
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   staid    6455 non-null   int64 
 1   staname  6455 non-null   object
 2   cn       6455 non-null   object
 3   lat      6455 non-null   object
 4   lon      6455 non-null   object
 5   hght     6455 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 302.7+ KB


In [33]:
# deleting all the dta from the table before loading (important for debuging)
with engine.begin() as conn:
    conn.execute(text("DELETE FROM stations;"))

In [35]:
# adding FK constraint for 'cn' column to countries.alpha2
with engine.begin() as conn:
    conn.execute(text("""
        ALTER TABLE stations
        ADD FOREIGN KEY (cn) REFERENCES countries (alpha2);
    """))

In [36]:
# loading 'stations' data from DataFrame to 'stations' table in 'climate' database
stations.to_sql('stations', engine, if_exists='append', index=False)

455

### Checking for missed data (value == -9999)

In [23]:
# Checking for the nulls (DataFrame NaN values)

stations.isnull().sum()

staid      0
staname    0
cn         0
lat        0
lon        0
hght       0
dtype: int64

In [25]:
# 'nulls' in 'staid'
# no nulls
mask_staid = (stations['staid'] == -9999)
stations[mask_staid]

,staid,staname,cn,lat,lon,hght


In [26]:
# 'nulls' in 'staname'
# no nulls
mask_staname = (stations['staname'] == '-9999')
stations[mask_staname]

,staid,staname,cn,lat,lon,hght


In [27]:
# 'nulls' in 'cn'
# no nulls
mask_cn = (stations['cn'] == '-9999')
stations[mask_cn]

,staid,staname,cn,lat,lon,hght


In [28]:
# 'nulls' in 'lat'
# no nulls
mask_lat = (stations['lat'] == '-9999')
stations[mask_lat]

,staid,staname,cn,lat,lon,hght


In [29]:
# 'nulls' in 'lon'
# no nulls
mask_lon = (stations['lon'] == '-9999')
stations[mask_lon]

,staid,staname,cn,lat,lon,hght


In [32]:
# 'nulls' in 'hght'
# 5 records with '-9999' value
mask_hght = (stations['hght'] == -9999)
stations[mask_hght]

,staid,staname,cn,lat,lon,hght
4788,18543,SLEIPNER B,NO,+58:25:04,+001:43:04,-9999
4956,18711,GRANEFELTET,NO,+59:09:55,+002:29:13,-9999
5006,18762,FRIGG,NO,+59:53:08,+002:04:15,-9999
5071,18829,YME,NO,+57:49:08,+004:31:10,-9999
5295,19066,HYWIND,NO,+59:16:59,+005:15:00,-9999


In [ ]:
### Checking for Primary Key (staid) uniqueness

In [24]:
# 'staid' is a Primary Key -> its values should be unique !!
# (number of unique values for 'staid' should be the same as total number of rows in DataFrame)
# PASSED
stations['staid'].nunique(), stations['staid'].count()

(6455, 6455)